In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import os
from openpyxl import load_workbook

In [3]:


#https://stackoverflow.com/questions/38074678/append-existing-excel-sheet-with-new-dataframe-using-python-pandas

def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,truncate_sheet=False, **to_excel_kwargs):
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [ ]:
# import requests
# choice='TCS'
# # "path":f"/api/option-chain-equities?symbol={choice}"
# while(True):
#     headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"gzip, deflate, br","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"https://www.nseindia.com/option-chain"}
#     url = "https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY"
#     page = requests.get(url,headers=headers)
    
#     if page.ok:
#         break
# page


In [26]:
import requests
headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
        'Sec-Fetch-User': '?1',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-Mode': 'navigate',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
    }

def nsefetch(payload):
    try:
        output = requests.get(payload,headers=headers).json()
        print(output)
    except ValueError:
        s =requests.Session()
        output = s.get("http://nseindia.com",headers=headers)
        output = s.get(payload,headers=headers).json()
    return output

res = nsefetch('https://www.nseindia.com/api/option-chain-equities?symbol=TCS')
res = res['records']['data']
res

-45.36333524064495

In [6]:
if len(res)>0:
    print('Yes')
else:
    print('no')

Yes


In [7]:
underlying_value = int(res[0]['PE']['underlyingValue'])
underlying_value

3115

In [8]:
nifty_margin = 500
bank_nifty_margin = 1000
stock_margin = 10

In [9]:
col_list = ['strikePrice','expiryDate','openInterest']
data_format = ['Date','Stock/Index','Expiry','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI']
base_df = pd.DataFrame(columns = data_format)
# append_df_to_excel("master.xlsx",base_df,sheet_name=str(i), index=False)

In [10]:
selected_margin = nifty_margin

In [11]:
# date = input("Enter date in format yyyy-mm-dd ")
date = '2021-05-27'

In [54]:
data = [d.get('CE') for d in res]
data_cleaned = list(filter(None, data))
df_call = pd.DataFrame(data_cleaned)

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)

df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)

# df_put['expiryDate'] = df_put['expiryDate'].dt.date
# df_call['expiryDate'] = df_call['expiryDate'].dt.date

df_put = df_put[col_list]
df_call = df_call[col_list]
# df_call
df_put

,strikePrice,expiryDate,openInterest
0,2400,2021-05-27,4
3,2500,2021-05-27,17
6,2600,2021-05-27,22
10,2650,2021-05-27,1
11,2700,2021-05-27,126
...,...,...,...
35,2950,2021-07-29,0
40,3000,2021-07-29,104
45,3050,2021-07-29,0
51,3100,2021-07-29,1


In [63]:
df_put = df_put[df_put['expiryDate'] == date]
df_call = df_call[df_call['expiryDate'] == date]
# df_put 
df_put['strikePrice'] = df_put['strikePrice'].map(int)
df_call['strikePrice'] = df_call['strikePrice'].map(int)

In [56]:
# print(res)

In [57]:
# min, max = ((100-percentage)*0.01)*(df['underlyingValue'][0]), ((100+percentage)*0.01)*(df['underlyingValue'][0])
# df_percent = df.loc[(df['strikePrice']>=min) & (df['strikePrice']<=max)]
# df_percent

In [58]:
selected_margin = underlying_value*0.1
selected_margin

311.5

In [59]:
df_put = df_put[(df_put['strikePrice'] >= (underlying_value-selected_margin)) & (df_put['strikePrice'] <= (underlying_value+selected_margin))]
df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
df_put['RIGHT'] = 'PUT'
df_call['RIGHT'] = 'CALL'

In [60]:
underlying_value-selected_margin

2803.5

In [61]:
max_sp = df_call.strikePrice.max()
min_sp = df_call.strikePrice.min()

In [62]:
sp_range = df_put.strikePrice.unique().tolist()
sp_range

[2820,
 2840,
 2850,
 2860,
 2880,
 2900,
 2920,
 2940,
 2950,
 2960,
 2980,
 3000,
 3020,
 3040,
 3050,
 3060,
 3080,
 3100,
 3120,
 3140,
 3150,
 3160,
 3180,
 3200,
 3220,
 3240,
 3250,
 3260,
 3280,
 3300,
 3320,
 3340,
 3350,
 3360,
 3380,
 3400,
 3420]

In [52]:
for i in sp_range:
    temp_df_put = df_put[df_put['strikePrice']==i]
    temp_df_call = df_call[df_call['strikePrice']==i]

In [53]:
temp1 = df_call
temp2 = df_put
# Ispe bharosa mat karna
temp2.merge(temp1,on='strikePrice')

,expiryDate_x,openInterest_x,RIGHT_x,strikePrice,expiryDate_y,openInterest_y,RIGHT_y


# Starting

In [ ]:
# date = input("Enter date in format yyyy-mm-dd ")


In [ ]:
df_put

In [ ]:
df_call

In [ ]:
df = df_put.merge(df_call, on='strikePrice')
# df['Time'] = ctime
df

In [ ]:
df['last_minute_OI_x'] = df['openInterest_x']
df['last_minute_OI_y'] = df['openInterest_y']
# df = df[['strikePrice','Time' ,'expiryDate_x', 'openInterest_x', 'last_minute_OI_x', 'RIGHT_x', 'openInterest_y', 'last_minute_OI_y', 'RIGHT_y']]
# df

# Keep this before

In [ ]:
import time
for i in sp_range:
    d = datetime.now().time()
    df['Time'] = d.strftime('%H:%M')
    append_df_to_excel("master.xlsx",df,sheet_name=str(i),header=None, index=False)

In [ ]:
# import pandas as pd
# import xlsxwriter

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

# df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

# writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

# df1.to_excel(writer, sheet_name='Sheet_1')

# df2.to_excel(writer, sheet_name='Sheet_2')

# df3.to_excel(writer, sheet_name='Sheet_3')

# writer.save()

# Initial Function - to be run once

In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import json


import requests

def main(time,previous_min_OI_x,previous_min_OI_y):
   
    while(True):
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
        url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
        page = requests.get(url,headers=headers)

        if page.ok:
            break

#     selected_margin = nifty_margin
   
    res = json.loads(page.text)['records']['data']
    underlying_value = int(res[0]['PE']['underlyingValue'])

    data = [d.get('CE') for d in res]
    data_cleaned = list(filter(None, data))
    df_call = pd.DataFrame(data_cleaned)

    data = [d.get('PE') for d in res]
    data_cleaned = list(filter(None, data))
    df_put = pd.DataFrame(data_cleaned)

    df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
    df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


    df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
    df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)
    
    col_list = ['strikePrice','expiryDate','openInterest']
    df_put = df_put[col_list]
    df_call = df_call[col_list]

    df_put = df_put[df_put['expiryDate'] == date]
    df_call = df_call[df_call['expiryDate'] == date]

    df_put['strikePrice'] = df_put['strikePrice'].map(int)
    df_call['strikePrice'] = df_call['strikePrice'].map(int)

    df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]
    df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
    df_put['RIGHT'] = 'PUT'
    df_call['RIGHT'] = 'CALL'
    df_merged = df_put.merge(df_call, on = 'strikePrice')
    df_merged.insert(0, 'Time', time)
    
#     df_merged[]
#     new_cols = ['Change_x', 'Percent_Change_x', 'Change_y', 'Percent_Change_y' ] 
#     df_merged[new_cols] = pd.DataFrame([[na.nan] * len(new_cols)], index=df.index)    

    cols = ['strikePrice','Time','Stock/Index','expiryDate_x','RIGHT_x','openInterest_x','previous_min_OI_x','change_OI_x',
                   '% Change_OI_x','RIGHT_y','openInterest_y','previous_min_OI_y','change_OI_y',
                   '% Change_OI_y']
    
#     df['last'] = dict(zip(df.ID, df.B))
#     print(df_merged.info())
#     print(df_merged)
#     df_merged.set_index('strikePrice',inplace=True)
###########################################################################

    # temp['Date'] = datetime.now().strftime('%Y%m%d')
    df_merged['Stock/Index'] = res[0]['PE']['underlying']
    df_merged['previous_min_OI_x'] = previous_min_OI_x
    df_merged['previous_min_OI_y'] = previous_min_OI_y
    df_merged['change_OI_x'] = df_merged['openInterest_x'] - df_merged['previous_min_OI_x']
    df_merged['change_OI_y'] =  df_merged['openInterest_y'] - df_merged['previous_min_OI_y']
    df_merged['% Change_OI_x'] = df_merged['change_OI_x']/df_merged['previous_min_OI_x'] * 100.0
    df_merged['% Change_OI_y'] = df_merged['change_OI_y']/df_merged['previous_min_OI_y'] * 100.0
    
    previous_min_OI_x = df_merged['openInterest_x'].values.tolist()
    previous_min_OI_y = df_merged['openInterest_y'].values.tolist()
        
#     df_merged = df_merged.style.format({'change_OI_x':"{0:+g}",'change_OI_y':"{0:+g}",'% Change_OI_x':"{0:+g}",'% Change_OI_y':"{0:+g}"})

###########################################################################
    
    df_merged = df_merged[cols]
    for i in sp_range:
        grouped = df_merged.groupby(df_merged.strikePrice)
        df_grouped = grouped.get_group(i)
         
        #added
        del df_grouped['strikePrice']
        
#         print(df_grouped)
        
        append_df_to_excel("master.xlsx",df_grouped,sheet_name=str(i), header=None, index=False)
    
    return previous_min_OI_x,previous_min_OI_y
    
    
    

In [ ]:
from datetime import datetime
import pandas as pd
import xlsxwriter

nifty_margin = 500
bank_nifty_margin = 1000
stock_margin = 10
date = '2021-05-27'
selected_margin = nifty_margin


while(True):
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
        url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
        page = requests.get(url,headers=headers)

        if page.ok:
            break

import json
res = json.loads(page.text)['records']['data']
underlying_value = int(res[0]['PE']['underlyingValue'])

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)
df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)

col_list = ['strikePrice','expiryDate','openInterest']

df_put = df_put[col_list]

df_put = df_put[df_put['expiryDate'] == date]

df_put['strikePrice'] = df_put['strikePrice'].map(int)

df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]

sp_range = df_put.strikePrice.unique().tolist()  

data_format = ['Time','Stock/Index','Expiry','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI']

base_df = pd.DataFrame(columns = data_format)

    
for i in sp_range:
    append_df_to_excel("master.xlsx",base_df,sheet_name=str(i), index=False, truncate_sheet = True)

# To be kept as backup
# global previous_min_OI_x = {i:0 for i in sp_range}
# global previous_min_OI_y = {i:0 for i in sp_range}

# global previous_min_OI_x 
previous_min_OI_x = [0 for i in sp_range]
# global previous_min_OI_y
previous_min_OI_y = [0 for i in sp_range]

while(True):
    now = datetime.now()
    current_time = now.strftime("%H:%M")
    print("Current Time =", current_time)
    previous_min_OI_x , previous_min_OI_y = main(current_time,previous_min_OI_x,previous_min_OI_y)
    time.sleep(60)

# BAckup Codes for References

In [ ]:
# global previous_min_OI_x = {i:np.nan for i in sp_range}
# global previous_min_OI_y = {i:np.nan for i in sp_range}
d = {el:np.nan for el in sp_range}
l = [0 for i in sp_range]
df

In [ ]:
dict = {'c':'B','z':'4'}

#mask those that are not NaN in `target_col`
m=df.target_col.isna()
df.loc[m,'target_col']=df.key_col.map(dict)

In [ ]:
# import pandas as pd
# import xlsxwriter

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

# df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

# writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

# df1.to_excel(writer, sheet_name='Sheet_1')

# df2.to_excel(writer, sheet_name='Sheet_2')

# df3.to_excel(writer, sheet_name='Sheet_3')

# writer.save()

In [ ]:
# import pandas as pd

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd'], 'D': ['a', 'bb', 'c', 'dd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4],  'D': ['a', 'bb', 'c', 'dd']})

# c = pd.concat([df1, df2], axis=1)
# c

In [ ]:
data_format = ['Time','Date','Stock/Index','Expiry','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI']
base_df = pd.DataFrame(columns = data_format)


In [ ]:
from copy import copy
temp = copy(df)
temp

In [ ]:
# temp.drop(['previous_min_OI_x','previous_min_OI_y','change_OI_x','change_OI_y','% Change_OI_x','% Change_OI_y'],axis=1,inplace=True)

In [ ]:
# temp['Date'] = datetime.now().strftime('%Y%m%d')
temp['Stock/Index'] = res[0]['PE']['underlying']
temp['previous_min_OI_x'] = previous_min_OI_x
temp['previous_min_OI_y'] = previous_min_OI_y
temp['change_OI_x'] = temp['openInterest_x'] - temp['previous_min_OI_x']
temp['change_OI_y'] =  temp['openInterest_y'] - temp['previous_min_OI_y']
temp['% Change_OI_x'] = temp['change_OI_x']/temp['previous_min_OI_x'] * 100.0
temp['% Change_OI_y'] = temp['change_OI_y']/temp['previous_min_OI_y'] * 100.0
temp = temp.style.format({'change_OI_x':"{0:+g}",'change_OI_y':"{0:+g}",'% Change_OI_x':"{0:+g}",'% Change_OI_y':"{0:+g}"})

In [ ]:
temp

In [ ]:
previous_min_OI_x = temp['openInterest_x'].values.tolist()
previous_min_OI_y = temp['openInterest_y'].values.tolist()

In [ ]:
from datetime import datetime
from time import time, sleep
while True:
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(current_time)
    sleep(60 - time() % 60)


In [ ]:
# IMPT Piece of code ! Dont delete
# df.style.format({'numbers':"{0:+g}"})

In [ ]:
from time import time, sleep
from datetime import datetime
while(True):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(current_time)
    sleep(83 - time() % 60)

16:35:34
16:36:23
